# ML pipeline

- Work in progress ...
- We will use tfx as a pipeline orchestrator

* [1. Data reading and splitting](#1)
* [2. Second title](#2)



In [13]:
import os
from tfx import v1 as tfx
import ml_metadata as mlmd

In [14]:
import tensorflow_model_analysis as tfma
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

In [15]:
from tfx.components.example_gen.csv_example_gen import component
from tfx.utils import proto_utils
from tfx.proto import example_gen_pb2
from tfx.v1 import proto

In [20]:
# Global variable for data relative path
_data_root = os.path.abspath("../data/inputs/train")

## 1. Data reading and splitting

In [23]:
# define a context for the pipeline
context = InteractiveContext(
  pipeline_name='decathlon_pipeline',
  pipeline_root='../deployment_pipeline'
  )

In [24]:
# Input has a single split 'input_dir/*'.
# Output 2 splits based on 'user_id' features: train:eval=3:1.

output = proto.Output(
             split_config=proto.SplitConfig(splits=[
                 proto.SplitConfig.Split(name='train', hash_buckets=10),
                 proto.SplitConfig.Split(name='eval', hash_buckets=1)
             ],
             partition_feature_name='day_id'))

example_gen = tfx.components.CsvExampleGen(input_base=_data_root, output_config=output)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [27]:
statistics_gen = tfx.components.StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen, enable_cache=True)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [28]:
context.show(statistics_gen.outputs['statistics'])

In [29]:
# Exclude the 'eval' split.
schema_gen = tfx.components.SchemaGen(
             statistics=statistics_gen.outputs['statistics'],
             exclude_splits=['eval'])

In [30]:
context.run(schema_gen, enable_cache=True)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [31]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'but_num_business_unit',INT,required,,-
'day_id',BYTES,required,,-
'dpt_num_department',INT,required,,-
'turnover',FLOAT,required,,-
